## 7주차 weekly challenge 주제
1. OpenAIEmbeddings 클래스를 사용해 문장 임베딩을 해보세요.

2. 사용자의 상황이나 배경을 제공해주면 그에 따라 이메일 양식을 만들어주는 챗봇을 만들어보세요.<br/>
ex) 거래처에 새해 인사와 함께 신규 계약건에 대해 이메일을 보내야한다.

3. 최신 뉴스를 브리핑 해주는 챗봇을 만들어보세요.

— 선택 —
4. 자신이 생각하는 가장 깔끔한 트랜스포머 모델을 파이토치로 구현하고 리얼한 데이터셋으로 학습과 추론을 해보세요.

5. 4번의 코드를 바탕으로 트랜스포머의 얼개를 설명하는 보고서를 작성해보세요.

## 1번 문제


```
# OpenAIEmbeddings 클래스를 사용해 문장 임베딩을 해보세요.
```



In [ ]:
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import re

class OpenAIEmbeddings:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)
        self.embeddings_cache = {}

    def embedding(self, query: str):
        if query not in self.embeddings_cache:
            response = self.client.embeddings.create(input=query, model="text-embedding-3-small")
            self.embeddings_cache[query] = response.data[0].embedding
        return self.embeddings_cache[query]

    def relevance(self, target_word: str, text: str):
        target_embedding = np.array(self.embedding(target_word)).reshape(1, -1)
        similarities = []

        words_list = re.split(r'[.,]+', text)
        words_list = [word.strip() for word in words_list if word.strip()]


        for word in words_list:
            try:
                word_embedding = np.array(self.embedding(word)).reshape(1, -1)
                similarity = cosine_similarity(target_embedding, word_embedding)[0][0]
                similarities.append((word, similarity))
            except Exception as e:
                print(f"Error processing word '{word}': {e}")

        return sorted(similarities, key=lambda x: x[1], reverse=True)[:10]

In [ ]:
# OpenAI API 키 설정 (환경 변수 또는 직접 설정)
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

model = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
target_word = "dinner"

text = """
In a small cabin deep in the mountains, I welcomed a peaceful morning.
Outside, the fog was thick, and warm sunlight seeped in faintly.
On the table were apples I picked myself from the forest yesterday,
bananas harvested from the backyard, and luscious oranges given by the neighbor's grandmother. The scent of freshly brewed coffee wafted softly, and I sat by the window, enjoying a leisurely breakfast.

Today is the day I've been dreaming of for a long time – the day I set off on a journey.
I packed my bags, left the cabin, and headed for the train station.
I boarded the old train, and as I looked out the window,
green forests and vast fields quickly passed by.
After a long ride, I arrived in the city center,
walked through grand buildings and bustling crowds, and finally reached the airport.

My heart pounded as I saw a huge airplane roaring into the sky.
When it was finally my turn, I boarded the plane and sat by the window, looking up at the sky.
The plane took off and soared above the clouds, and the world below looked like a toy.
The vibrant streets of a strange city, delicious food, and friendly people
– this trip will remain a memorable experience for a long time.
"""

# 코사인 유사도 계산 및 결과 출력
similarities = model.relevance(target_word, text)

# 유사도 값 확인 및 출력
found = False
for word, similarity in similarities:
    if similarity > 0.3:
        print(f"{word}: {similarity:.4f}")
        found = True

if not found:
    print("0.3을 넘는 유사도 값이 없습니다.")

delicious food: 0.4631
enjoying a leisurely breakfast: 0.3734


## 2번 문제


```
# 사용자의 상황이나 배경을 제공해주면 그에 따라 이메일 양식을 만들어주는 챗봇을 만들어보세요.
ex) 거래처에 새해 인사와 함께 신규 계약건에 대해 이메일을 보내야한다.
```



In [ ]:
from openai import OpenAI
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

def email_generation(user, receiver, backgroud):
    info = {
        'start' : "글 전체의 주제를 찾아내 '주제 : '와 같이 글의 맨 위에 나타내고, 인사말은 '안녕하세요! ~~ receiver 이름 님'과 같이 시작 하라.",
        'ask': "사용자의 정보와 배경정보를 가지고서 이메일 형식으로 글을 작성하라. 제공한 정보들을 사용하여 완성된 이메일 글을 작성하라. 그리고 가능하면 점(.)뒤에서 줄바꿈 할수 있도록 적절하게 줄바꿈을 하여 글의 가독성을 올려라. 받는 사람 이름을 명시하지 말고 모든 내용을 추론하여 완성된 글을 완성해야 한다. 글은 한글로 작성하라.",
        'user': user,
        'receiver' : receiver,
        'back': backgroud,
    }

    prompt = f"""
        아래의 제공한 정보들을 바탕으로 글을 완성하라. 가장 중요한 것은 글의 가독성과 전달력이다.
        최초로 작성된 문장을 다시 한번 한눈에 볼 수 있도록(너무 문장이 길지 않도록) 글을 다듬은 다음 출력하라.

        글의 시작 : {info['start']}

        요청내용 : {info['ask']}

        사용자 정보: {info['user']}

        받는 사람 : {info['receiver']}

        배경 정보: {info['back']}
    """

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "assistant",
                "content": prompt
            }
        ]
    )

    print(completion.choices[0].message.content)

In [ ]:
user = "구름 마케팅 기업의 Haebo / 직무: 외주를 받아와 작업을 처리"
receiver = "(주)파이토치 코퍼레이션"
backgroud = "거래처에 2025년 새해 인사와 함께 신규 계약건에 대해 이메일을 보내야한다. 받는 사람에게 마케팅을 위한 홍보 유튜브 채널 운영이 2025년 3월에 시작되었음을 알려야 한다."

email_generation(user, receiver, backgroud)

주제: 2025년 새해 인사 및 신규 계약건 안내

---

안녕하세요, 파이토치 코퍼레이션 님.

구름 마케팅의 Haebo입니다.
새해를 맞아 귀사에 따뜻한 인사를 전하고 싶습니다.
2025년이 새로운 기회와 성과로 가득한 한 해가 되시기를 기원합니다.

저희 구름 마케팅은 귀사와의 지속적인 협력을 진심으로 소중하게 생각하고 있으며, 2025년에도 변함없이 최선을 다할 것을 약속드립니다.
또한, 이번 기회에 신규 계약건에 대해 안내드리고자 합니다.

저희는 귀사의 성공적인 마케팅 활동을 지원하기 위해 새로운 홍보 유튜브 채널을 2025년 3월부터 운영하기로 하였습니다.
이 채널은 귀사의 브랜드 인지도를 증대시키고 소비자와의 소통을 더욱 강화하는 데 큰 역할을 할 것입니다.

새해에도 변함없이 귀사의 마케팅 파트너로서 최선을 다할 수 있도록 하겠습니다.
언제든지 추가 정보가 필요하시거나 문의사항이 있으시면 편하게 연락 주시기 바랍니다.

감사합니다. 

Haebo 드림
구름 마케팅



## 3번 문제


```
# 최신 뉴스를 브리핑 해주는 챗봇을 만들어보세요
```



In [ ]:
# !pip install tavily-python
# !pip install langchain_openai
from tavily import TavilyClient
from langchain_openai import ChatOpenAI
from google.colab import userdata
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

class news_briefing_model:
    def __init__(self):
        self.tavily = TavilyClient(api_key = TAVILY_API_KEY)
        self.gpt_4o = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o", temperature=0.1)
        self.results = None
        self.briefing = None

    def latest_news(self,query):
        search_response = self.tavily.search(
                query=query,
                search_type="advanced",
                time_range="week"
            )

        results = search_response.get("results", [])
        self.results = results[:5]

    def news_briefing(self):
        prompt = f"""
            최신 뉴스기사들을 실제 'title', 'content', 'url'을 사용하여 보기 좋게 정리해 드립니다.
            '오늘의 뉴스 브리핑 시작합니다.'로 시작하여 최대한 부드러운 말투로 글을 작성해드립니다.
            주어진 데이터를 기반으로 뉴스 브리핑을 진행해 주세요!

            뉴스 정보 : {self.results}
            """
        response = self.gpt_4o.invoke(prompt)
        self.briefing = response.content

    def __call__(self, news = "뉴스"):
        self.latest_news(news)
        self.news_briefing()
        return self.briefing

model = news_briefing_model()
news = "오늘의 날씨"
model(news)

print(model.briefing)

오늘의 뉴스 브리핑 시작합니다.

1. **[오늘의 날씨] 구름 많다가 오전부터 맑아져...큰 일교차 유의 - 한국면세뉴스**
   - 오늘은 전국적으로 구름이 많다가 오전부터 맑아질 예정입니다. 특히 큰 일교차가 예상되니 건강 관리에 유의하시기 바랍니다. 수도권 등 일부 지역에서는 미세먼지 농도가 '나쁨' 수준을 보일 것으로 예상되니 외출 시 주의가 필요합니다.
   - [기사 읽기](https://www.kdfnews.com/news/articleView.html?idxno=152227)

2. **03월 19일 수요일, 오늘의 날씨와 미세먼지 농도 [인포그래픽] - 시선뉴스**
   - 오늘은 중국 상하이 부근에 위치한 고기압의 가장자리에 들면서 전국적으로 가끔 구름이 많다가 오전부터 맑아질 것으로 보입니다. 서울의 경우 최저기온 -2도, 최고기온 7도, 미세먼지 농도는 '보통' 수준입니다.
   - [기사 읽기](https://www.sisunnews.co.kr/news/articleView.html?idxno=222395)

3. **[오늘 날씨] 전국 맑고 바람 강해…낮 최고 서울 7도·부산 10도 - 뉴스웍스**
   - 오늘 아침 최저 기온은 -7~2도, 낮 최고 기온은 6~11도로 예보되었습니다. 평년보다 2~6도가량 낮은 기온을 보일 것으로 예상되며, 바람도 강하게 불어 체감 온도가 더 낮을 수 있으니 따뜻한 옷차림이 필요합니다.
   - [기사 읽기](https://www.newsworks.co.kr/news/articleView.html?idxno=788456)

4. **[오늘날씨] 전국에 눈 또는 비…아침 기온도 '영하권' - 아이뉴스24**
   - 오늘은 전국적으로 눈 또는 비가 예상되며, 일부 지역에서는 돌풍과 천둥·번개도 예상됩니다. 수도권과 전라권을 시작으로 밤에는 대부분 그칠 것으로 보이며, 제주도 산지는 내일 새벽까지 이어질 수 있습니다.
   - [기사 읽기](https://www.inews24.com/view/18241

In [ ]:
# CLI에서 챗봇 구현하기
model = news_briefing_model()
cnt = 3
while cnt > 0:
    user_input = input("알고 싶은 최신 뉴스 키워드를 입력해주세요! ex) 날씨, 주식 등 : ")
    response = model(user_input)
    cnt -= 1
    print(response)

알고 싶은 최신 뉴스 키워드를 입력해주세요! : 판교역
오늘의 뉴스 브리핑 시작합니다.

1. **8201번 버스 시간표 및 노선 안내**
   - **내용 요약**: 8201번 버스는 안성 버스터미널에서 출발하여 한경대, 중앙대, 판교역, 야탑역을 경유합니다. 특히 판교역은 신분당선이 지나며, IT 기업 종사자들의 출퇴근이 편리한 위치에 있습니다. 야탑역은 분당선이 지나며, 주변에 다양한 편의시설이 위치해 있습니다.
   - **자세히 보기**: [링크](https://rndlife.tistory.com/2700)

2. **서울역에서 판교까지의 경로 안내**
   - **내용 요약**: 서울역에서 판교까지의 교통편에 대한 정보입니다. 소요 시간, 환승 횟수, 승차 노선, 정차역 목록 등 자세한 경로 안내를 제공합니다.
   - **자세히 보기**: [링크](https://transit.navitime.com/ko/kr/transfer?start=00000531&goal=00000616)

3. **판교역 장항선 열차 시간표**
   - **내용 요약**: 판교역 장항선에서 운행하는 모든 열차의 운행 시간 및 요금을 안내합니다. 전국 KTX 및 열차/기차 운행 정보도 함께 제공됩니다.
   - **자세히 보기**: [링크](https://train.asamaru.net/시간표/판교역-장항선/)

4. **화성 6003번 버스 노선 및 시간표**
   - **내용 요약**: 화성 6003번 버스는 동탄2차고지에서 출발하여 판교역, 낙생육교, 현대백화점까지 운행합니다. 직행좌석버스로, 시내버스 요금이 적용됩니다.
   - **자세히 보기**: [링크](https://bustrain.kr/citybus/10112)

오늘의 뉴스 브리핑은 여기까지입니다. 편안한 하루 보내세요!
알고 싶은 최신 뉴스 키워드를 입력해주세요! : 김치의 효능
오늘의 뉴스 브리핑 시작합니다.

1. **김치볶음밥 맛있게 만드는 법 | 황금레시피ㅣ칼로리**
   - 김치볶음밥을 맛있게 만드

## 4번 문제(연습)

In [140]:
import torch
import torch.nn as nn
import numpy as np

# data 준비
data = [
    [1, 2, 3, 4, 5, 6, 7, 8, 9],
    [2, 3, 4, 5, 6, 7, 8, 9, 10],
    [3, 4, 5, 6, 7, 8, 9, 10, 11],
]
data = torch.tensor(data, dtype = torch.float32)

# 처음 8개의 값으로 뒤의 8개의 값을 예측하는 모델을 만들기 위한 것
inputs = data[:, :-1] # 마지막 col 값 뺌
targets = data[:, 1:] # 첫번째 col 값 뺌

In [143]:
# 모델 정의
class SimpleTransformer(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
        super(SimpleTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim, output_dim)
        self.transformer = nn.Transformer(d_model = model_dim,
                                          nhead= num_heads,
                                          num_encoder_layers=num_layers,
                                          num_decoder_layers=num_layers,
                                          batch_first= True)  # (시퀀스 길이, 배치 크기, 특성 차원) -> (배치 크기, 시퀀스 길이, 특성 차원)
        self.fc_out = nn.Linear(model_dim, output_dim)

    def forward(self, src, tgt): # src = 인코더의 입력, tgt = 디코더의 입력
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        # permute() : tensor의 차원 순서를 변경 : (시퀀스 길이, 배치 크기, 특성 차원) -> (배치 크기, 시퀀스 길이, 특성 차원)
        output = self.transformer(src.permute(1, 0, 2), tgt.permute(1, 0, 2))
        output = self.fc_out(output.permute(1, 0, 2))
        return output

# 모델 입력 변수 값 설정
input_dim = 1 # 입력 차원
model_dim = 64 # 모델의 차원(임베딩 차원)
num_heads = 2 # multi-head attention heads 개수
num_layers = 2 # encoder-decoder layers 개수
output_dim = 1 # 출력 차원

# 모델 인스턴스 생성
model = SimpleTransformer(input_dim, model_dim, num_heads, num_layers, output_dim)

In [144]:
# 모델 컴파일(Compilation) : loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train model
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(inputs.unsqueeze(-1),inputs.unsqueeze(-1))
    loss = criterion(output.squeeze(-1), targets)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 50 == 0:  # 로그 빈도 조정
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)

## 4번 문제



```
# 자신이 생각하는 가장 깔끔한 트랜스포머 모델을 파이토치로 구현하고 리얼한 데이터셋으로 학습과 추론을 해보세요.
```
[self-attention paper : attention is all you need](https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)
